In [1]:
dir_meta = '../data/skin-cancer'
dir_data = './outputs/gradcam_guided_images'
dir_ckpt = './outputs/ckpt'

n_seeds = 15
n_classes = 7
input_shape = (28, 28, 1)

epochs = 10

In [2]:
import sys
sys.path.append('../')

from cnntdanet.utils import seed_all
from cnntdanet.models import get_cnn_net
from cnntdanet.utils.plotting import plot_learning_curve

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [3]:
def get_2d_cnn(input_shape, n_classes, name='img_network'):
    cnn = keras.models.Sequential([
        keras.layers.Conv2D(16, 3, activation="relu", padding='same', input_shape=input_shape),
        keras.layers.Conv2D(16, 3, activation="relu", padding='same'),
        keras.layers.AveragePooling2D(2),
        keras.layers.Conv2D(32, 3, activation="relu", padding='same'),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.3),
        keras.layers.Conv2D(64, 3, activation="relu", padding='same'),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.3),
        keras.layers.Conv2D(128, 3, activation="relu", padding='same'),
        keras.layers.MaxPooling2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(7, activation='softmax')
    ], name=name)

    return cnn


In [4]:
df = pd.read_csv(os.path.join(dir_meta, 'hmnist_28_28_L.csv'))
y = df['label'].values.astype(np.int64)
y_ = y.copy()
y = tf.keras.utils.to_categorical(y_, n_classes)

result_cnn = []
result_cnn_tda = []
for seed in range(n_seeds):
    # Splitting data
    train_indices, valid_indices = train_test_split(np.arange(len(y)), stratify=y_, test_size=0.2, random_state=seed)
    
    # Data preparation
    X = np.load(os.path.join(dir_data, 'cnn.npy')) 
    X_train, y_train = X[train_indices], y[train_indices]
    X_valid, y_valid = X[valid_indices], y[valid_indices]
    
    seed_all(seed)
    cnn = get_2d_cnn(input_shape=input_shape, n_classes=n_classes)
    cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    history = cnn.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=epochs,
        verbose=False
    )
    y_true = np.argmax(y_valid, axis=1) # From one-hot encoding to label
    y_pred = cnn.predict(X_valid)  # Make predictions
    y_pred = np.argmax(y_pred, axis=1)  # Take the maximum probability for each prediction
    result_cnn.append([
        accuracy_score(y_true, y_pred),
        recall_score(y_true, y_pred, average='macro'),
        precision_score(y_true, y_pred, average='weighted', zero_division=0),
        f1_score(y_true, y_pred, average='weighted', zero_division=0)
    ])
    ###########################################################################################################
    # Data preparation
    X = np.load(os.path.join(dir_data, 'cnn_tda.npy')) 
    X_train, y_train = X[train_indices], y[train_indices]
    X_valid, y_valid = X[valid_indices], y[valid_indices]
    
    seed_all(seed)
    cnn = get_2d_cnn(input_shape=input_shape, n_classes=n_classes)
    cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    history = cnn.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=epochs,
        verbose=False
    )
    y_true = np.argmax(y_valid, axis=1) # From one-hot encoding to label
    y_pred = cnn.predict(X_valid)  # Make predictions
    y_pred = np.argmax(y_pred, axis=1)  # Take the maximum probability for each prediction
    result_cnn_tda.append([
        accuracy_score(y_true, y_pred),
        recall_score(y_true, y_pred, average='macro'),
        precision_score(y_true, y_pred, average='weighted', zero_division=0),
        f1_score(y_true, y_pred, average='weighted', zero_division=0)
    ])

2023-05-12 19:30:06.051717: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
np.array(result_cnn).mean(axis=0)

array([0.70870361, 0.2570574 , 0.64409733, 0.65691026])

In [6]:
np.array(result_cnn_tda).mean(axis=0)

array([0.7027126 , 0.24582475, 0.64004072, 0.64281327])